In [ ]:

create or replace table `yhcr-prd-bradfor-bia-core.CB_2172.cb_2172_efi2_pev_alc_harm` as (
#this identifies the care home cohort
with b as (
  SELECT person_id, episodestartdate
  from `yhcr-prd-bradfor-bia-core.CB_2172.care_home_cohort_v1`),

# this code selects all snomed codes for the care home cohort individuals
person as (
    SELECT 
  person_id,
  snomedcode,
  dateevent
  FROM`yhcr-prd-bradfor-bia-core.CB_FDM_PrimaryCare.tbl_srcode`
  WHERE person_id in(SELECT person_id FROM b)),

prev_harm as (
  SELECT 
    person.person_id,
    person.snomedcode,
    person.dateevent as ph_date,
    b.episodestartdate as ch_entry_date
    from person 
    INNER JOIN  b
    on person.person_id = b.person_id
  WHERE person.snomedcode in(SELECT SNOMEDCT_CONCEPTID from `yhcr-prd-bradfor-bia-core.CB_LOOKUPS.tbl_EFI2_Codelist` where Deficit= 'Alcohol' AND Otherinstructions = 'Previous higher risk/harmful drinking')),
  
  prev_harm_no as (
  Select 
  *,
  row_number() over (partition by person_id order by ph_date desc) as prev_harm_seq
  from prev_harm
  Where ph_date < ch_entry_date AND date_diff(prev_harm.ch_entry_date,prev_harm.ph_date,year) <=5 ),

  prev_harm_filtered as (
    SELECT distinct
    *,
    case when ph_date IS NOT NULL then 'previous higher risk/harmful drinking' end as alcohol_status
    from prev_harm_no
    where prev_harm_seq = 1
    order by person_id, ph_date)

    select 
    person_id,
    prev_harm_filtered.alcohol_status
    FROM prev_harm_filtered)


